<a href="https://colab.research.google.com/github/pmy0792/pytorch_tutorial/blob/main/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multivariate Linear Regression

In [1]:
import torch
from torch import optim

In [3]:
# 데이터
x_train=torch.FloatTensor([[73,80,75],
                           [93,88,93],
                           [89,91,90],
                           [96,98,100],
                           [73,66,70]])
y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

# 모델 초기화
W=torch.zeros((3,1),requires_grad=True)
b=torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b],lr=1e-5)


In [5]:
nb_epochs=20
for epoch in range(nb_epochs +1):

  # H(x) 계산
  hypothesis = x_train.matmul(W) + b

  # cost 계산 (MSE)
  cost=torch.mean((hypothesis - y_train)**2)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step() #updates parameter

  print('Epoch {:4d}/{}  hypothesis: {} Cost: {:.6f}'.format(
      epoch,nb_epochs, hypothesis.squeeze().detach(), # detach: 기존 Tensor에서 gradient 전파가 안되는 텐서 생성
      cost.item()
  ))

Epoch    0/20  hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    1/20  hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    2/20  hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    3/20  hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    4/20  hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371063
Epoch    5/20  hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    6/20  hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    7/20  hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    8/20  hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch    9/20  hypothesis: tensor([152.3454, 183.1161, 180.4231, 196.4765, 139.6732

## nn.Module을 사용하여 모델 생성

In [7]:
import torch.nn as nn

In [ ]:
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(3,1)

  def forward(self,x): # hypothesis 계산
    return self.linear(x)

hypothesis=model(x_train)

## torch.nn.functional의 loss function 사용

In [13]:
import torch.nn.functional as F

# 쉽게 다른 loss와 교체 가능
#cost = F.mse_loss(prediction,y_train)

In [19]:
# 데이터
x_train=torch.FloatTensor([[73,80,75],
                           [93,88,93],
                           [89,91,90],
                           [96,98,100],
                           [73,66,70]])
y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

# 모델 초기화
model=MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr=1e-5)


# 학습
nb_epochs = 20
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  hypothesis = model(x_train)

  # cost 계산
  cost = F.mse_loss(hypothesis,y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{}  hypothesis: {} Cost: {:.6f}'.format(
      epoch,nb_epochs, hypothesis.squeeze().detach(), # detach: 기존 Tensor에서 gradient 전파가 안되는 텐서 생성
      cost.item()
  ))


Epoch    0/20  hypothesis: tensor([42.1518, 51.6074, 50.4483, 54.3867, 39.9636]) Cost: 15421.927734
Epoch    1/20  hypothesis: tensor([ 90.6490, 109.8979, 107.8827, 116.9313,  84.4246]) Cost: 4834.392578
Epoch    2/20  hypothesis: tensor([117.8007, 142.5327, 140.0380, 151.9477, 109.3169]) Cost: 1515.763550
Epoch    3/20  hypothesis: tensor([133.0018, 160.8037, 158.0406, 171.5521, 123.2532]) Cost: 475.549805
Epoch    4/20  hypothesis: tensor([141.5123, 171.0332, 168.1196, 182.5278, 131.0558]) Cost: 149.497040
Epoch    5/20  hypothesis: tensor([146.2768, 176.7603, 173.7623, 188.6727, 135.4242]) Cost: 47.297050
Epoch    6/20  hypothesis: tensor([148.9442, 179.9668, 176.9215, 192.1130, 137.8701]) Cost: 15.262662
Epoch    7/20  hypothesis: tensor([150.4375, 181.7620, 178.6902, 194.0390, 139.2395]) Cost: 5.221419
Epoch    8/20  hypothesis: tensor([151.2734, 182.7672, 179.6803, 195.1174, 140.0063]) Cost: 2.073836
Epoch    9/20  hypothesis: tensor([151.7413, 183.3301, 180.2347, 195.7210, 140.4

## Pytorch Dataset, DataLoader

In [20]:
from torch.utils.data import Dataset

In [35]:
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data=[[73,80,75],
                 [93,88,93],
                 [89,91,90],
                 [96,98,100],
                 [73,66,70]]
    self.y_data=[[152],[185],[180],[196],[142]]
  
  def __len__(self):
    return len(self.x_data)

  def __getitem__(self,idx):
    x=torch.FloatTensor(self.x_data[idx])
    y=torch.FloatTensor(self.y_data[idx])
    return x,y

dataset=CustomDataset()

In [22]:
from torch.utils.data import DataLoader

In [36]:
dataloader = DataLoader(
    dataset,
    batch_size=2, # 각 minibatch의 크기
    shuffle=True, # epoch마다 데이터셋 섞어서 데이터가 학습되는 순서를 바꾼다
)

In [38]:
model=MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(),lr=1e-5)


nb_epochs=20
for epoch in range(nb_epochs+1):
  for batch_idx, samples in enumerate(dataloader):
    x_train,y_train=samples

    # H(x) 계산
    prediction=model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{}  Batch: {}/{} Cost: {:.6f}'.format(
      epoch,nb_epochs, batch_idx+1, len(dataloader), # detach: 기존 Tensor에서 gradient 전파가 안되는 텐서 생성
      cost.item()
  ))


Epoch    0/20  Batch: 1/3 Cost: 21048.320312
Epoch    0/20  Batch: 2/3 Cost: 16734.615234
Epoch    0/20  Batch: 3/3 Cost: 3035.065674
Epoch    1/20  Batch: 1/3 Cost: 653.142944
Epoch    1/20  Batch: 2/3 Cost: 393.204010
Epoch    1/20  Batch: 3/3 Cost: 43.578411
Epoch    2/20  Batch: 1/3 Cost: 26.893318
Epoch    2/20  Batch: 2/3 Cost: 9.918231
Epoch    2/20  Batch: 3/3 Cost: 1.189956
Epoch    3/20  Batch: 1/3 Cost: 1.946731
Epoch    3/20  Batch: 2/3 Cost: 0.142445
Epoch    3/20  Batch: 3/3 Cost: 1.457182
Epoch    4/20  Batch: 1/3 Cost: 0.167780
Epoch    4/20  Batch: 2/3 Cost: 0.697871
Epoch    4/20  Batch: 3/3 Cost: 0.950191
Epoch    5/20  Batch: 1/3 Cost: 0.847172
Epoch    5/20  Batch: 2/3 Cost: 0.298989
Epoch    5/20  Batch: 3/3 Cost: 0.122932
Epoch    6/20  Batch: 1/3 Cost: 0.797054
Epoch    6/20  Batch: 2/3 Cost: 0.034976
Epoch    6/20  Batch: 3/3 Cost: 0.482039
Epoch    7/20  Batch: 1/3 Cost: 0.180369
Epoch    7/20  Batch: 2/3 Cost: 0.343475
Epoch    7/20  Batch: 3/3 Cost: 1.896000